In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
seed = 42
tf.random.set_seed(seed)

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
x_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'LUMO / eV']
X = data[x_cols]
y = data[col_names[2]]

In [5]:
X_scaled = MinMaxScaler(feature_range=(0,1)).fit_transform(X)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=seed)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation='relu', input_shape=(len(x_cols),)),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1)
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,541
Trainable params: 1,541
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.005),
    loss='mean_squared_error')

In [10]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 55ms/step - loss: 0.4506 - val_loss: 0.3464
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 0.3186 - val_loss: 0.2330
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 0.2100 - val_loss: 0.1534
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1335 - val_loss: 0.1133
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1102 - val_loss: 0.1063
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1122 - val_loss: 0.1004
Epoch 7/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1111 - val_loss: 0.0827
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0947 - val_loss: 0.0654
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0806 - val_loss: 0.0604
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0739 - val_loss: 0.0616
Epoch 11/100
2/2 [==========

Epoch 84/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0228 - val_loss: 0.0215
Epoch 85/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0229 - val_loss: 0.0223
Epoch 86/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0226 - val_loss: 0.0211
Epoch 87/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0230 - val_loss: 0.0210
Epoch 88/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0226 - val_loss: 0.0233
Epoch 89/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0228 - val_loss: 0.0251
Epoch 90/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0226 - val_loss: 0.0229
Epoch 91/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0219 - val_loss: 0.0218
Epoch 92/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0219 - val_loss: 0.0211
Epoch 93/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0221 - val_loss: 0.0211
Epoch 94/100
2/2 [==